In [1]:
import sys
import os
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import tqdm
from tqdm.notebook import tqdm
import re
import urllib.request

In [30]:
data = pd.read_excel('moviedata.xlsx')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8238 entries, 0 to 8237
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   순번      8238 non-null   int64         
 1   영화명     8238 non-null   object        
 2   감독      8100 non-null   object        
 3   개봉일     8159 non-null   datetime64[ns]
 4   국적      8238 non-null   object        
 5   전국관객수   8238 non-null   int64         
 6   장르      8236 non-null   object        
 7   등급      8238 non-null   object        
dtypes: datetime64[ns](1), int64(2), object(5)
memory usage: 515.0+ KB


In [4]:
data = data[data['전국관객수'] > 1000000]

In [5]:
data.drop([6585, 6180], axis=0, inplace=True)

In [6]:
data

,순번,영화명,감독,개봉일,국적,전국관객수,장르,등급
145,574,정직한 후보,장유정,2020-02-12,한국,1525718,코미디,12세이상관람가
152,672,클로젯,김광빈,2020-02-05,한국,1264970,미스터리,15세이상관람가
171,128,남산의 부장들,우민호,2020-01-22,한국,4750104,드라마,15세이상관람가
172,344,히트맨,최원섭,2020-01-22,한국,2406232,코미디,15세이상관람가
184,693,해치지않아,손재곤,2020-01-15,한국,1209689,코미디,12세이상관람가
...,...,...,...,...,...,...,...,...
7557,517,품행제로,조근식,2002-12-26,한국,1694637,코미디,15세관람가
7560,106,반지의 제왕 : 두개의 탑,피터 잭슨,2002-12-19,미국,5183000,액션,12세관람가
7563,167,해리포터와 비밀의 방,크리스 콜럼버스,2002-12-13,미국,4000000,드라마,전체관람가
7565,161,색즉시공,윤제균,2002-12-11,한국,4082797,코미디,18세관람가


In [7]:
list(data['영화명'])[0]

'정직한 후보'

In [2]:
#query_txt = list(data['영화명'])
start_date="20030620"
end_date="20200718"
#q = '"영화"'

In [8]:
path = 'chromedriver.exe'
#options = webdriver.ChromeOptions()
#options.add_argument('headless')    # 웹 브라우저를 띄우지 않는 headless chrome 옵션 적용
#options.add_argument('disable-gpu')    # GPU 사용 안함
#options.add_argument('lang=ko_KR')    # 언어 설정, options=options
#driver = webdriver.Chrome('chromedriver', options=options)



driver = webdriver.Chrome(path)#, options=options)
driver.get('http://www.naver.com')
time.sleep(2)
# 네이버 검색창에 검색어 검색
element = driver.find_element_by_id('query')
element.send_keys("영화 ", list(data['영화명'])[0])  # query_txt는 위에서 입력한 '영화 리뷰'
element.submit()

# 블로그 카테고리 선택
driver.find_element_by_link_text('블로그').click()

# # 검색옵션 버튼 클릭

driver.find_element_by_id('_search_option_btn').click()
    
#날짜 입력
driver.find_element_by_xpath("""//*[@id="snb"]/div/ul/li[2]/a""").click()
time.sleep(2)

#날짜 입력
s_date = driver.find_element_by_xpath("""//*[@id="blog_input_period_begin"]""")     
driver.find_element_by_xpath("""//*[@id="blog_input_period_begin"]""").click()
s_date.clear()  # 날짜 입력 부분에 기존에 입력되어 있던 날짜를 제거합니다. 
time.sleep(1)
    
# 아래 코드가 날짜를 for 반복문으로 1 글자씩 입력
for c in start_date:
    s_date.send_keys(c)
    time.sleep(0.1)

# 종료 날짜 입력
e_date = driver.find_element_by_xpath("""//*[@id="blog_input_period_end"]""")
driver.find_element_by_xpath("""//*[@id="blog_input_period_end"]""").click()
e_date.clear()
time.sleep(1)

for c in end_date:
    e_date.send_keys(c)
    time.sleep(0.1)

driver.find_element_by_class_name('tx').click()
time.sleep(3)
    
driver.find_element_by_xpath("""//*[@id="snb"]/div/ul/li[1]""").click()
driver.find_element_by_xpath("""//*[@id="snb"]/div/ul/li[1]/div/ul/li[1]/a""").click()
time.sleep(1)
    
    
url_list = []
title_list = []
title_list2 = []
# 몇개의 페이지를 크롤링 할지 선택

total_page = 1
for j in tqdm(list(data['영화명'])[0:]) : 
    for i in range(total_page) : # 페이지 번호
        i = i * 1 
        url = "https://search.naver.com/search.naver?date_from={0}&date_option=8&date_to={1}&dup_remove=1&nso=p%3Afrom{2}to{3}&post_blogurl=&post_blogurl_without=&query=영화%20{4}%20줄거리&sm=tab_pge&srchby=all&st=sim&where=post&start={5}".format(start_date,end_date,start_date,end_date, j, i)
        driver.get(url)
        time.sleep(0.5)
        
# URL 크롤링 시작
    titles = "a.sh_blog_title._sp_each_url._sp_each_title"
    moive_raw = driver.find_elements_by_css_selector(titles)
    count = 0
    # url 크롤링 시작    
    for movie in moive_raw[:4]: # url 4개만 긁음
        url = movie.get_attribute('href')
        url_list.append(url)
# #         count += 1
# #         if url == '' :
# #             url_list.append('0')
# #             print('0개')
# #         else :
# #             url_list.append(url)

#     print(count, url)
     # 제목   
    for movie in moive_raw[:4]: # 타이틀도 4개만 긁어옴
        title = movie.get_attribute('title')   
        title_list2.append(title)    

        
for i in list(data['영화명'])[0:] :
    for j in range(1, 5) :
        title_list.append(i)

    
print('url갯수: ', len(url_list))
print('title갯수: ', len(title_list))



df = pd.DataFrame({'url':url_list, 'title':title_list2})

# # 저장하기
df.to_csv("blog_url2.csv", encoding='utf-8-sig')


url_load = pd.read_csv("blog_url2.csv", index_col=0)        # 기본 모델
num_list = len(url_load)
    

dict = {}  # 전체 크롤링 데이터 저장

# ★수집할 글 갯수
#number = url_list
for i in tqdm(range(2903, num_list)) : 
    # 글 띄우기
    url = url_load['url'][i]
    driver = webdriver.Chrome('chromedriver.exe')#, options=options)
    driver.get(url)   # 글 띄우기
    
#     # 크롤링
    
    try : 
        # iframe 접근
        driver.switch_to_frame('mainFrame')

        target_info = {}

        # 제목 크롤링 시작
        overlays = ".se-fs-.se-ff-"                                 
        tit = driver.find_element_by_css_selector(overlays)         # title
        title = tit.text
        title
        
        # 내용 크롤링
        overlays = ".se-component.se-text.se-l-default"                                 
        contents = driver.find_elements_by_css_selector(overlays)         # date

        content_list = []
        for content in contents:
            content_list.append(content.text)

        content_str = ' '.join(content_list)

        # 글 하나는 target_info
        target_info['title'] = title
        target_info['content'] = content_str

        # 각각의 글은 dict라는 딕셔너리에.
        dict[i] = target_info
        time.sleep(1)
        
#         print(i, title)

        # 글 하나 크롤링 후 크롬 창 닫기
        driver.close()       
    
    # 에러나면 현재 크롬창 닫고 다음 글(i+1)로 이동
    except: 
        driver.close()
        time.sleep(1)
        continue
    
#     # 중간,중간에 파일로 저장하기
    if i == 100 or 2000 or 4000:
        # 판다스로 만들기
        import pandas as pd
        result_df = pd.DataFrame.from_dict(dict, 'index')

        # 저장하기
        result_df.to_csv("blog_content_movie3.csv", encoding='utf-8-sig')
        time.sleep(3)

# print('수집한 글 갯수: ', len(dict))
# # print(dict)

# 판다스로 만들기
import pandas as pd
result_df = pd.DataFrame.from_dict(dict, 'index')
result_df.to_csv("blog_content_movie3.csv", encoding='utf-8-sig')

# df2 = pd.read_csv('blog_content_movie3.csv', index_col=0)
# a = pd.DataFrame(title_list)
# df2['title2'] = a
# df2.to_csv('blog_content_movie3.csv', encoding='utf-8-sig')


url갯수:  3188
title갯수:  3188


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:127: DeprecationWarning: use driver.switch_to.frame instead


# 합치는 작업

In [27]:
movie1 = pd.read_csv('C:\Python37_Project\FinalProject\영화\새 폴더\\blog_content_movie3.csv', index_col=0)
movie2 = pd.read_csv('C:\Python37_Project\FinalProject\영화\새 폴더\\blog_content_movie4.csv', index_col=0)
movie3 = pd.read_csv('C:\Python37_Project\FinalProject\영화\새 폴더\\blog_content_movie5.csv', index_col=0)

In [56]:
movie_final = pd.concat([movie1, movie2, movie3], axis=0)

In [57]:
movie_final

,title,content
1,https://news.joins.com/article/23707085,[영화] 정직한 후보 배우 줄거리 후기 쿠키영상 #영화 #정직한후보 를 #조조영화...
2,NaN,요즘 선거철이라 그런지 그에 맞게 영화 정직한후보를\n엄청 홍보하는 것 같더라구요...
3,NaN,"진실의 주둥이가 터진다!\n정직한 후보\nHONEST CANDIDATE, 2019\..."
4,안녕하세요 배재대학교,안녕하세요 배재대학교 \n홍보대사 20기 최주은입니다.\n다들 건강관리는 잘 하시고...
5,"갈 곳이 옷장안 밖에 없는, 사랑 받지 못한 아이들의 이야기-공포 영화 클로젯 결말...",올해 첫 공포 영화가 개봉을 했다. 영화 보기 그리 좋은 시절은 아니다. 신종코로...
...,...,...
3181,영화 손오공 4탄 색즉시공 사묘 뤄샤오이 판뤼주연 줄거리 결말예측,"안녕하세요? 용이나르샤 입니다. ""손오공 4탄 : 색즉시공"" 소개합니다. 손오공 역..."
3183,[추천 영화],또 주인공 인원이 바뀌었네...삼장법사 서양인으로 나옴. <줄거리>\n사막을 걷다가...
3184,액션 로맨스영화 추천 광복절 특사 줄거리,오늘은 액션 로맨스영화 추천 광복절 특사 줄거리를 이야기 해보도록 할께요.\n이 영...
3186,<영화> 광복절 특사,김상진 감독의 코미디 영화다. 이전 작품에 연을 맺었던 차승원이 주연을 맡았다. 그...


In [66]:
df2 = movie_final
a = pd.DataFrame(title_list)
df2['title2'] = a

In [175]:
df2.to_csv('movie_final.csv', encoding='utf-8-sig', index=True)

In [176]:
mv = pd.read_csv('movie_final.csv')

In [177]:
mv.drop('Unnamed: 0', inplace=True, axis=1)

In [178]:
mv.to_csv('movie_final.csv', encoding='utf-8-sig')

In [179]:
movie_last = mv.pivot_table(index='title2', values='content', aggfunc=sum)

In [180]:
movie_last

,content
title2,
007 스카이폴,이번시리즈 영화는 많은 의미가 부여되어진 작품이기도한데요 시리즈가 탄생한지 벌...
007 스펙터,오늘 새 핸드폰을 구매 하러 신도림 테크노 마트를 방문 하였습니다. 그런데 주차장에...
007 퀀텀 오브 솔러스,2008년에 개봉한 영화이네요 벌써 이시리즈를 기준으로 \n22번째의 작품이면서...
1987,1979년 12·12사태로 정권을 잡은 전두환 군사정권의 장기집권을 저지하기 위해 ...
1번가의 기적,영화 1번가의기적줄거리 1000만관객을 동원한 영화를 만들엇던 그의 과거작품이구요 ...
...,...
후궁 : 제왕의 첩,후궁:제왕의 첩\n장르-멜로/로멘스 드라마\n개봉-2012.06.06(한국 122분...
흡혈형사 나도열,오늘은 배꼽빠지도록 웃긴 영화 흡혈형사 나도열의 줄거리 및 감상평을 적어볼까 해요....
히말라야,이미 개봉했을 당시 영화관에서 본 작품이지만\n추천작에 뜨길래 또 보고 싶어져서...


In [181]:
movie_data = data.merge(movie_last, how='left', left_on='영화명', right_on='title2')

In [182]:
movie_data1 = movie_data[movie_data['content'].notnull()]

In [183]:
movie_data1.to_csv('movie_data1.csv', encoding='utf-8-sig')

# 누락된 영화 75개 찾기

In [15]:
movie_data = pd.read_csv('movie_data.csv', index_col=0)

In [16]:
movie_data

,순번,영화명,감독,개봉일,국적,전국관객수,장르,등급,content
0,574,정직한 후보,장유정,2020-02-12,한국,1525718,코미디,12세이상관람가,[영화] 정직한 후보 배우 줄거리 후기 쿠키영상 #영화 #정직한후보 를 #조조영화...
1,672,클로젯,김광빈,2020-02-05,한국,1264970,미스터리,15세이상관람가,안녕하세요 배재대학교 \n홍보대사 20기 최주은입니다.\n다들 건강관리는 잘 하시고...
2,128,남산의 부장들,우민호,2020-01-22,한국,4750104,드라마,15세이상관람가,역사와 진실이 항상 승리하고 이길 수 있는 것은 역사는 어떤 사람이나 집단에 휘둘리...
3,344,히트맨,최원섭,2020-01-22,한국,2406232,코미디,15세이상관람가,안녕하세요. 카엔할부 임실장입니다.\n여러분 잘 지내고 계시나요?\n슬기롭게 블태기...
4,693,해치지않아,손재곤,2020-01-15,한국,1209689,코미디,12세이상관람가,"해치지 않아\nSecret Zoo, 2019 영화 해치지않아 줄거리, 결말 있음. ..."
...,...,...,...,...,...,...,...,...,...
792,517,품행제로,조근식,2002-12-26,한국,1694637,코미디,15세관람가,영화가 좋다 시동 품행제로 터미네이터 다크페이트 6 언더그라운드 나를 찾아줘 지푸라...
793,106,반지의 제왕 : 두개의 탑,피터 잭슨,2002-12-19,미국,5183000,액션,12세관람가,영화 반지의제왕:두개의탑\n감독 : #피터잭슨\n출연 : #일라이저우드 #이안맥켈런...
794,167,해리포터와 비밀의 방,크리스 콜럼버스,2002-12-13,미국,4000000,드라마,전체관람가,"해리포터 시리즈의 두번째 영화인 비밀의 방에는 SNS에서 종종 쓰이는 ""도비는 이제..."
795,161,색즉시공,윤제균,2002-12-11,한국,4082797,코미디,18세관람가,오늘은 한국 청불 영화 추천 색즉시공 줄거리를 이야기 해볼께요.\n색즉시공의 주연은...


In [17]:
missing_data = movie_data[movie_data['content'].isnull()]

In [18]:
missing_data

,순번,영화명,감독,개봉일,국적,전국관객수,장르,등급,content
39,536,걸캅스,정다원,2019-05-09,한국,1625037,코미디,15세이상관람가,NaN
54,453,내안의 그놈,강효진,2019-01-09,한국,1901684,판타지,15세이상관람가,NaN
71,415,명당,박희곤,2018-09-19,한국,2087474,사극,12세이상관람가,NaN
77,114,공작,윤종빈,2018-08-08,한국,4971261,드라마,12세이상관람가,NaN
81,70,미션 임파서블: 폴아웃,크리스토퍼 맥쿼리,2018-07-25,미국,6584915,액션,15세이상관람가,NaN
...,...,...,...,...,...,...,...,...,...
740,752,령,"김태경,이인규",2004-06-17,한국,1108769,공포(호러),12세관람가,NaN
755,23,실미도,강우석,2003-12-24,한국,11081000,전쟁,15세관람가,NaN
770,720,툼레이더 2 : 판도라의 상자,얀 드봉,2003-08-01,미국,1165906,액션,15세관람가,NaN
782,558,와일드 카드,김유진,2003-05-16,한국,1571931,범죄,18세관람가,NaN


In [35]:
# path = 'chromedriver.exe'
# #options = webdriver.ChromeOptions()
# #options.add_argument('headless')    # 웹 브라우저를 띄우지 않는 headless chrome 옵션 적용
# #options.add_argument('disable-gpu')    # GPU 사용 안함
# #options.add_argument('lang=ko_KR')    # 언어 설정, options=options
# #driver = webdriver.Chrome('chromedriver', options=options)



# driver = webdriver.Chrome(path)#, options=options)
# driver.get('http://www.naver.com')
# time.sleep(2)
# # 네이버 검색창에 검색어 검색
# element = driver.find_element_by_id('query')
# element.send_keys("영화 ", list(missing_data['영화명'])[0])  # query_txt는 위에서 입력한 '영화 리뷰'
# element.submit()

# # 블로그 카테고리 선택
# driver.find_element_by_link_text('블로그').click()

# # # 검색옵션 버튼 클릭

# driver.find_element_by_id('_search_option_btn').click()
    
# #날짜 입력
# driver.find_element_by_xpath("""//*[@id="snb"]/div/ul/li[2]/a""").click()
# time.sleep(2)

# #날짜 입력
# s_date = driver.find_element_by_xpath("""//*[@id="blog_input_period_begin"]""")     
# driver.find_element_by_xpath("""//*[@id="blog_input_period_begin"]""").click()
# s_date.clear()  # 날짜 입력 부분에 기존에 입력되어 있던 날짜를 제거합니다. 
# time.sleep(1)
    
# # 아래 코드가 날짜를 for 반복문으로 1 글자씩 입력
# for c in start_date:
#     s_date.send_keys(c)
#     time.sleep(0.1)

# # 종료 날짜 입력
# e_date = driver.find_element_by_xpath("""//*[@id="blog_input_period_end"]""")
# driver.find_element_by_xpath("""//*[@id="blog_input_period_end"]""").click()
# e_date.clear()
# time.sleep(1)

# for c in end_date:
#     e_date.send_keys(c)
#     time.sleep(0.1)

# driver.find_element_by_class_name('tx').click()
# time.sleep(3)
    
# driver.find_element_by_xpath("""//*[@id="snb"]/div/ul/li[1]""").click()
# driver.find_element_by_xpath("""//*[@id="snb"]/div/ul/li[1]/div/ul/li[1]/a""").click()
# time.sleep(1)
    
    
# url_list = []
# title_list = []
# title_list2 = []
# # 몇개의 페이지를 크롤링 할지 선택

# total_page = 1
# for j in tqdm(list(missing_data['영화명'])[0:]) : 
#     for i in range(total_page) : # 페이지 번호
#         i = i * 1 
#         url = "https://search.naver.com/search.naver?date_from={0}&date_option=8&date_to={1}&dup_remove=1&nso=p%3Afrom{2}to{3}&post_blogurl=&post_blogurl_without=&query=영화%20{4}%20줄거리&sm=tab_pge&srchby=all&st=sim&where=post&start={5}".format(start_date,end_date,start_date,end_date, j, i)
#         driver.get(url)
#         time.sleep(0.5)
        
# # URL 크롤링 시작
#     titles = "a.sh_blog_title._sp_each_url._sp_each_title"
#     moive_raw = driver.find_elements_by_css_selector(titles)
#     count = 0
#     # url 크롤링 시작    
#     for movie in moive_raw[5:11]: # url 5개만 긁음
#         url = movie.get_attribute('href')
#         url_list.append(url)
# # #         count += 1
# # #         if url == '' :
# # #             url_list.append('0')
# # #             print('0개')
# # #         else :
# # #             url_list.append(url)

# #     print(count, url)
#      # 제목   
#     for movie in moive_raw[5:11]: # 타이틀도 15개만 긁어옴
#         title = movie.get_attribute('title')   
#         title_list2.append(title)    

        
# for i in list(missing_data['영화명'])[0:] :
#     for j in range(1, 6) :
#         title_list.append(i)

    
# print('url갯수: ', len(url_list))
# print('title갯수: ', len(title_list))



# df = pd.DataFrame({'url':url_list, 'title':title_list2})

# # # 저장하기
# df.to_csv("blog_url2.csv", encoding='utf-8-sig')


# url_load = pd.read_csv("blog_url2.csv", index_col=0)        # 기본 모델
# num_list = len(url_load)
    

# dict = {}  # 전체 크롤링 데이터 저장

# # ★수집할 글 갯수
# #number = url_list
# for i in tqdm(range(0, num_list)) : 
#     # 글 띄우기
#     url = url_load['url'][i]
#     driver = webdriver.Chrome('chromedriver.exe')#, options=options)
#     driver.get(url)   # 글 띄우기
    
# #     # 크롤링
    
#     try : 
#         # iframe 접근
#         driver.switch_to_frame('mainFrame')

#         target_info = {}

#         # 제목 크롤링 시작
#         overlays = ".se-fs-.se-ff-"                                 
#         tit = driver.find_element_by_css_selector(overlays)         # title
#         title = tit.text
#         title
        
#         # 내용 크롤링
#         overlays = ".se-component.se-text.se-l-default"                                 
#         contents = driver.find_elements_by_css_selector(overlays)         # date

#         content_list = []
#         for content in contents:
#             content_list.append(content.text)

#         content_str = ' '.join(content_list)

#         # 글 하나는 target_info
#         target_info['title'] = title
#         target_info['content'] = content_str

#         # 각각의 글은 dict라는 딕셔너리에.
#         dict[i] = target_info
#         time.sleep(1)
        
# #         print(i, title)

#         # 글 하나 크롤링 후 크롬 창 닫기
#         driver.close()       
    
#     # 에러나면 현재 크롬창 닫고 다음 글(i+1)로 이동
#     except: 
#         driver.close()
#         time.sleep(1)
#         continue
    
# #     # 중간,중간에 파일로 저장하기
#     if i == 100 or 2000 or 4000:
#         # 판다스로 만들기
#         import pandas as pd
#         result_df = pd.DataFrame.from_dict(dict, 'index')

#         # 저장하기
#         result_df.to_csv("missing_data.csv", encoding='utf-8-sig')
#         time.sleep(3)

# print('수집한 글 갯수: ', len(dict))
# # print(dict)

# 판다스로 만들기
import pandas as pd
result_df = pd.DataFrame.from_dict(dict, 'index')
result_df.to_csv("missing_data.csv", encoding='utf-8-sig')

# df2 = pd.read_csv('blog_content_movie3.csv', index_col=0)
# a = pd.DataFrame(title_list)
# df2['title2'] = a
# df2.to_csv('blog_content_movie3.csv', encoding='utf-8-sig')

In [37]:
result_df['title2'] = a

In [39]:
result_df.drop([372,373], inplace=True, axis=0)

In [40]:
result_df

,title,content,title2
0,"Miss & Mrs. Cops, 2018",걸크러시 콤비의 비공식\n합동 수사가 펼쳐진다!\n걸캅스\nMiss & Mrs. C...,걸캅스
1,"이번주 볼만한 영화 알라딘,","이번주 볼만한 영화 알라딘,\n걸캅스, 악인전 줄거리 \n내일은 문화의날~ 안녕하세...",걸캅스
2,킬링타임용 영화 추천 걸캅스 줄거리,오늘은 킬링타임용 영화 걸캅스 줄거리를 이야기 해볼까합니다.\n요즘 김사부2에서 의...,걸캅스
3,,"영화, <걸캅스> 관람과 줄거리\n 2019년 5월 12일(일) 주일이며 부처님 오...",걸캅스
4,영화 걸캅스 줄거리 및 결말,"이번 포스팅은 라미란, 이성경 주연의 영화 걸캅스입니다.\n먼저 줄거리 부터, 한 ...",걸캅스
...,...,...,...
335,"한국의 공포영화추천 레전드 6편[여고괴담, 기담, 알포인트, 장화홍련, 불신지옥, ...","다즐패키지에서 추천해 드리는 공포영화추천, 한국의 공포영화 레전드 6편\n여고괴담,...",분신사바
355,,"강우석 감독의 실미도는 한국 영화 최초로 1,000만(1108만) 관객을 동원한 영...",실미도
356,<초간단줄거리> 김일성을 죽이기 위해 사형수들을 지옥훈련 시키지만 필요 없게 되자 ...,<초간단줄거리> 김일성을 죽이기 위해 사형수들을 지옥훈련 시키지만 필요 없게 되자 ...,실미도
359,"실미도 (Silmido, 2003) - 영화 다시보기","전쟁, 스릴러, 드라마, 액션 한국 135분 2003 .12.24 개봉 [국내] 1...",실미도


In [52]:
result_df = result_df.pivot_table(index='title2', values='content', aggfunc=sum)

In [59]:
s2 = missing_data.merge(result_df, how='inner', left_on='영화명', right_on='title2')
s2.drop('content_x', inplace=True, axis=1)
s2 = s2.rename(columns={'content_y':'content'})

In [60]:
s2

,순번,영화명,감독,개봉일,국적,전국관객수,장르,등급,content
0,536,걸캅스,정다원,2019-05-09,한국,1625037,코미디,15세이상관람가,걸크러시 콤비의 비공식\n합동 수사가 펼쳐진다!\n걸캅스\nMiss & Mrs. C...
1,453,내안의 그놈,강효진,2019-01-09,한국,1901684,판타지,15세이상관람가,"#내안의그놈줄거리 #내안의그놈결과\n내안의 그놈 (Inside me, 2019) ..."
2,415,명당,박희곤,2018-09-19,한국,2087474,사극,12세이상관람가,"명당, 관상 시리즈 궁 밖에 살던 송화옹주가 궁에 들어온다.\n부마 간택을 위해 서..."
3,114,공작,윤종빈,2018-08-08,한국,4971261,드라마,12세이상관람가,영화 공작 줄거리 요약 및 결말 스포 \n 영화 공작 TMI급 자세한 줄거리 및 결...
4,70,미션 임파서블: 폴아웃,크리스토퍼 맥쿼리,2018-07-25,미국,6584915,액션,15세이상관람가,"액션, 모험, 스릴러 영화인 미션임파서블 폴아웃!\n미션임파서블 시리즈는 평점이 9..."
5,710,바람 바람 바람,이병헌,2018-04-05,한국,1183959,코미디,청소년관람불가,안녕하세요 잇님들 벌써 금욜이네요! 불금~이 왓도다!!! \n주말계획 다들 세우셨나...
6,54,1987,장준환,2017-12-27,한국,7231638,드라마,15세이상관람가,오랜만에 가슴이 먹먹해지는 영화\n한편 소개해 드리려고 해요.\n그때 그 시절로 돌...
7,593,부라더,장유정,2017-11-02,한국,1468301,코미디,12세이상관람가,어제 밤새면서 넷플릭스로 보게 된 영화2\n#부라더\n이동휘 배우 좋아하는데 이건 ...
8,64,범죄도시,강윤성,2017-10-03,한국,6851297,범죄,청소년관람불가,안녕하세요 여러분들 \nㅎㅎㅎㅎ\n훈이찡입니다~ 오늘은 월요일!\n벌서 이렇게 첫...
9,772,"날, 보러와요",이철하,2016-04-07,한국,1061412,미스터리,청소년관람불가,"날, 보러와요 는 강예원, 이상윤 주연의 2015년 개봉한 영화입니다. 요새 케이..."


In [63]:
movie_data1

,순번,영화명,감독,개봉일,국적,전국관객수,장르,등급,content
0,574,정직한 후보,장유정,2020-02-12,한국,1525718,코미디,12세이상관람가,[영화] 정직한 후보 배우 줄거리 후기 쿠키영상 #영화 #정직한후보 를 #조조영화...
1,672,클로젯,김광빈,2020-02-05,한국,1264970,미스터리,15세이상관람가,안녕하세요 배재대학교 \n홍보대사 20기 최주은입니다.\n다들 건강관리는 잘 하시고...
2,128,남산의 부장들,우민호,2020-01-22,한국,4750104,드라마,15세이상관람가,역사와 진실이 항상 승리하고 이길 수 있는 것은 역사는 어떤 사람이나 집단에 휘둘리...
3,344,히트맨,최원섭,2020-01-22,한국,2406232,코미디,15세이상관람가,안녕하세요. 카엔할부 임실장입니다.\n여러분 잘 지내고 계시나요?\n슬기롭게 블태기...
4,693,해치지않아,손재곤,2020-01-15,한국,1209689,코미디,12세이상관람가,"해치지 않아\nSecret Zoo, 2019 영화 해치지않아 줄거리, 결말 있음. ..."
...,...,...,...,...,...,...,...,...,...
792,517,품행제로,조근식,2002-12-26,한국,1694637,코미디,15세관람가,영화가 좋다 시동 품행제로 터미네이터 다크페이트 6 언더그라운드 나를 찾아줘 지푸라...
793,106,반지의 제왕 : 두개의 탑,피터 잭슨,2002-12-19,미국,5183000,액션,12세관람가,영화 반지의제왕:두개의탑\n감독 : #피터잭슨\n출연 : #일라이저우드 #이안맥켈런...
794,167,해리포터와 비밀의 방,크리스 콜럼버스,2002-12-13,미국,4000000,드라마,전체관람가,"해리포터 시리즈의 두번째 영화인 비밀의 방에는 SNS에서 종종 쓰이는 ""도비는 이제..."
795,161,색즉시공,윤제균,2002-12-11,한국,4082797,코미디,18세관람가,오늘은 한국 청불 영화 추천 색즉시공 줄거리를 이야기 해볼께요.\n색즉시공의 주연은...


In [64]:
s2

,순번,영화명,감독,개봉일,국적,전국관객수,장르,등급,content
0,536,걸캅스,정다원,2019-05-09,한국,1625037,코미디,15세이상관람가,걸크러시 콤비의 비공식\n합동 수사가 펼쳐진다!\n걸캅스\nMiss & Mrs. C...
1,453,내안의 그놈,강효진,2019-01-09,한국,1901684,판타지,15세이상관람가,"#내안의그놈줄거리 #내안의그놈결과\n내안의 그놈 (Inside me, 2019) ..."
2,415,명당,박희곤,2018-09-19,한국,2087474,사극,12세이상관람가,"명당, 관상 시리즈 궁 밖에 살던 송화옹주가 궁에 들어온다.\n부마 간택을 위해 서..."
3,114,공작,윤종빈,2018-08-08,한국,4971261,드라마,12세이상관람가,영화 공작 줄거리 요약 및 결말 스포 \n 영화 공작 TMI급 자세한 줄거리 및 결...
4,70,미션 임파서블: 폴아웃,크리스토퍼 맥쿼리,2018-07-25,미국,6584915,액션,15세이상관람가,"액션, 모험, 스릴러 영화인 미션임파서블 폴아웃!\n미션임파서블 시리즈는 평점이 9..."
5,710,바람 바람 바람,이병헌,2018-04-05,한국,1183959,코미디,청소년관람불가,안녕하세요 잇님들 벌써 금욜이네요! 불금~이 왓도다!!! \n주말계획 다들 세우셨나...
6,54,1987,장준환,2017-12-27,한국,7231638,드라마,15세이상관람가,오랜만에 가슴이 먹먹해지는 영화\n한편 소개해 드리려고 해요.\n그때 그 시절로 돌...
7,593,부라더,장유정,2017-11-02,한국,1468301,코미디,12세이상관람가,어제 밤새면서 넷플릭스로 보게 된 영화2\n#부라더\n이동휘 배우 좋아하는데 이건 ...
8,64,범죄도시,강윤성,2017-10-03,한국,6851297,범죄,청소년관람불가,안녕하세요 여러분들 \nㅎㅎㅎㅎ\n훈이찡입니다~ 오늘은 월요일!\n벌서 이렇게 첫...
9,772,"날, 보러와요",이철하,2016-04-07,한국,1061412,미스터리,청소년관람불가,"날, 보러와요 는 강예원, 이상윤 주연의 2015년 개봉한 영화입니다. 요새 케이..."


In [67]:
movie_final2 = pd.concat([movie_data1,s2], ignore_index=True)

In [68]:
movie_final2.to_csv('블로그최종.csv', encoding='utf-8-sig', index=True)

In [69]:
movie_final2

,순번,영화명,감독,개봉일,국적,전국관객수,장르,등급,content
0,574,정직한 후보,장유정,2020-02-12,한국,1525718,코미디,12세이상관람가,[영화] 정직한 후보 배우 줄거리 후기 쿠키영상 #영화 #정직한후보 를 #조조영화...
1,672,클로젯,김광빈,2020-02-05,한국,1264970,미스터리,15세이상관람가,안녕하세요 배재대학교 \n홍보대사 20기 최주은입니다.\n다들 건강관리는 잘 하시고...
2,128,남산의 부장들,우민호,2020-01-22,한국,4750104,드라마,15세이상관람가,역사와 진실이 항상 승리하고 이길 수 있는 것은 역사는 어떤 사람이나 집단에 휘둘리...
3,344,히트맨,최원섭,2020-01-22,한국,2406232,코미디,15세이상관람가,안녕하세요. 카엔할부 임실장입니다.\n여러분 잘 지내고 계시나요?\n슬기롭게 블태기...
4,693,해치지않아,손재곤,2020-01-15,한국,1209689,코미디,12세이상관람가,"해치지 않아\nSecret Zoo, 2019 영화 해치지않아 줄거리, 결말 있음. ..."
...,...,...,...,...,...,...,...,...,...
742,795,데이지,유위강,2006-03-09,한국,1023096,멜로/로맨스,15세관람가,#데이지의맛 - 사라진 소녀를 찾아라 !! 복수 & 정의 ?!\n#넷플릭스데이지의맛...
743,734,"안녕, 형아",임태형,2005-05-27,한국,1143136,드라마,전체관람가,어느덧 6월의 첫날! 첫날부터 \n새로 시작하는 드라마가 있습니다. \n바로 #한예...
744,809,분신사바,안병기,2004-08-05,한국,1001125,공포(호러),15세관람가,"다즐패키지에서 추천해 드리는 공포영화추천, 한국의 공포영화 레전드 6편\n여고괴담,..."
745,23,실미도,강우석,2003-12-24,한국,11081000,전쟁,15세관람가,"강우석 감독의 실미도는 한국 영화 최초로 1,000만(1108만) 관객을 동원한 영..."
